Using the BBC collection in question, combine the 1 and -1 classes into one and perform classification using the SVM (linear kernel) method, comparing the subjective class (1 and -1) with the objective class (0). Conduct the research for reasonably similar numbers of posts from each class (e.g., 500 and 500). 

In [5]:
# importing libraries
import pandas as pd
import numpy as np
import tmtoolkit as tm
from sklearn.model_selection import train_test_split
from sklearn import svm

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib.pyplot as plt

import kagglehub
from kagglehub import KaggleDatasetAdapter

c:\Users\paula\OneDrive\Pulpit\git_swiezosc\NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
file_path = "DATASET.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "alexandrakim2201/spotify-dataset",
  file_path
)

df['label'] = df['label'].map({'POSITIVE': 1, 'NEGATIVE': 0})
df = df.sort_values(by = 'label')
df

C:\Users\paula\AppData\Local\Temp\ipykernel_15372\293340492.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,Review,label
26350,I have discovered a new world of music. Before...,0
24369,Won't play music. It was my fav music app and ...,0
24368,Watch a ad u will get 30 minutes ad free music...,0
41902,Really like this app. But even when it tells m...,0
24365,I don't know what happend but I can't see my s...,0
...,...,...
22812,Listening songs is my hobby . And spotify help...,1
22811,Very good music experience.,1
22810,"Yes,it is a nice music appðŸ‘ŒðŸ‘Œ",1
22841,Smooth running app,1
